# <span style="color:#F72585"><center>Generación de texto usando caracteres y redes recurrentes </center></span>

<center>Introducción</center>

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Tratamiento_de_Lenguaje_Natural/Imagenes/maquina_de_escribir.jpg" width="600" height="600" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Muestreo  de palabras</p>
</figcaption>
</figure>

Fuente: [Pexels](https://www.pexels.com/es-es/foto/reescribir-y-editar-texto-en-una-maquina-de-escribir-3631711/)

##   <span style="color:#4361EE">Introducción</span>

Este cuaderno es una adaptación del tutorial de tensorflow [Text generation with a RNN](https://www.tensorflow.org/tutorials/text/text_generation).

Se muestra como generar texto usando una RNR basado en caracteres. Trabajaremos con un conjunto de datos de los algunos documentos de cuentos disponibles en la [biblioteca libre Gutemberg](https://www.gutenberg.org/), con los códigos:  

+ 55514-0.txt, 
+ 61244-0.txt, 
+ pg36805.txt, 
+ pg45438.txt, 
+ pg46000.txt, 
+ 30053-0.txt,

Y los poemas de Daniel Montenegro, uno de los autores:

+ Poemas_Output.txt
+ Poemas_Todo.txt

Los datos fueron preparados por Alvaro Montenegro, uno de los autores.

Dada una secuencia de caracteres a partir de estos datos  se entrena un modelo para predecir el siguiente caracter en la secuencia. 

Se pueden generar secuencias de texto más largas llamando al modelo repetidamente.

## <span style="color:#4361EE">Importa módulos requeridos</span>

In [2]:
import tensorflow as tf

import numpy as np
import os
import time
import glob

print("Versión de Tensorflow: ", tf.__version__)

Versión de Tensorflow:  2.8.2


##   <span style="color:#4361EE">Lee los datos</span>

Obtiene el `path` en donde están los datos y los lee en una única lista.

In [3]:
path = '../Datos/RNR/'
files = glob.glob(path+ '*.txt')
files

['../Datos/RNR/61244-0.txt',
 '../Datos/RNR/55514-0.txt',
 '../Datos/RNR/pg46000.txt',
 '../Datos/RNR/Poemas_Output.txt',
 '../Datos/RNR/Poemas_Todo.txt',
 '../Datos/RNR/pg45438.txt',
 '../Datos/RNR/pg36805.txt']

In [4]:
text = []
for file in files:
    t = open(files[6], 'rb').read().decode(encoding='utf-8')
    text.extend(t)

In [5]:
len(text)

1334361

### <span style="color:#4CC9F0">Una mirada a los primeros 250 caracteres</span>

In [6]:
# Echa un vistazo a los primeros 250 caracteres del texto
print(text[:250])

['\r', '\n', 'L', 'O', 'S', ' ', 'C', 'O', 'N', 'S', 'E', 'J', 'O', 'S', ' ', 'D', 'E', ' ', 'U', 'N', ' ', 'P', 'A', 'D', 'R', 'E', '\r', '\n', '\r', '\n', '\r', '\n', 'E', 'l', ' ', 'L', 'e', 'ó', 'n', ',', ' ', 'e', 'l', ' ', 'r', 'e', 'y', ' ', 'd', 'e', ' ', 'l', 'a', 's', ' ', 's', 'e', 'l', 'v', 'a', 's', ',', ' ', 'a', 'g', 'o', 'n', 'i', 'z', 'a', 'b', 'a', ' ', 'e', 'n', ' ', 'e', 'l', ' ', 'h', 'u', 'e', 'c', 'o', ' ', 'd', 'e', ' ', 's', 'u', ' ', 'c', 'a', 'v', 'e', 'r', 'n', 'a', '.', '.', '.', '.', '\r', '\n', '\r', '\n', 'Á', ' ', 's', 'u', ' ', 'l', 'a', 'd', 'o', ' ', 'e', 's', 't', 'a', 'b', 'a', ' ', 's', 'u', ' ', 'h', 'i', 'j', 'o', ',', ' ', 'e', 'l', ' ', '_', 'n', 'u', 'e', 'v', 'o', ' ', 'l', 'e', 'ó', 'n', '_', ',', ' ', 'e', 'l', ' ', 'r', 'e', 'y', ' ', 'f', 'u', 't', 'u', 'r', 'o', ' ', 'd', 'e', ' ', 't', 'o', 'd', 'o', 's', ' ', 'l', 'o', 's', '\r', '\n', 'a', 'n', 'i', 'm', 'a', 'l', 'e', 's', '.', '\r', '\n', '\r', '\n', 'E', 'l', ' ', 'm', 'o', 'n', '

In [7]:
text[:50]

['\r',
 '\n',
 'L',
 'O',
 'S',
 ' ',
 'C',
 'O',
 'N',
 'S',
 'E',
 'J',
 'O',
 'S',
 ' ',
 'D',
 'E',
 ' ',
 'U',
 'N',
 ' ',
 'P',
 'A',
 'D',
 'R',
 'E',
 '\r',
 '\n',
 '\r',
 '\n',
 '\r',
 '\n',
 'E',
 'l',
 ' ',
 'L',
 'e',
 'ó',
 'n',
 ',',
 ' ',
 'e',
 'l',
 ' ',
 'r',
 'e',
 'y',
 ' ',
 'd',
 'e']

##   <span style="color:#4361EE">Crea  el alfabeto</span>

Este alfabeto es una lista que contiene las letras mayúsculas y minúsculas, y algunos caracteres especiales, incluyendo el salto de línea `\n`.

In [8]:
# Los caracteres únicos en el archivo.
vocab = sorted(set(text))
print ('{} caracteres únicos'.format(len(vocab)))

83 caracteres únicos


In [9]:
vocab

['\n',
 '\r',
 ' ',
 '!',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¡',
 '«',
 '»',
 '¿',
 'Á',
 'É',
 'Í',
 'Ñ',
 'Ó',
 'á',
 'é',
 'í',
 'ï',
 'ñ',
 'ó',
 'ú',
 'ü']

##   <span style="color:#4361EE">Crea los diccionarios </span>

- Caracter a índice
- Índice a caracter

In [10]:
# Creación de un mapeo de caracteres únicos a índices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [11]:
char2idx

{'\n': 0,
 '\r': 1,
 ' ': 2,
 '!': 3,
 '(': 4,
 ')': 5,
 '*': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'L': 23,
 'M': 24,
 'N': 25,
 'O': 26,
 'P': 27,
 'Q': 28,
 'R': 29,
 'S': 30,
 'T': 31,
 'U': 32,
 'V': 33,
 'X': 34,
 'Y': 35,
 'Z': 36,
 '[': 37,
 ']': 38,
 '_': 39,
 'a': 40,
 'b': 41,
 'c': 42,
 'd': 43,
 'e': 44,
 'f': 45,
 'g': 46,
 'h': 47,
 'i': 48,
 'j': 49,
 'k': 50,
 'l': 51,
 'm': 52,
 'n': 53,
 'o': 54,
 'p': 55,
 'q': 56,
 'r': 57,
 's': 58,
 't': 59,
 'u': 60,
 'v': 61,
 'w': 62,
 'x': 63,
 'y': 64,
 'z': 65,
 '¡': 66,
 '«': 67,
 '»': 68,
 '¿': 69,
 'Á': 70,
 'É': 71,
 'Í': 72,
 'Ñ': 73,
 'Ó': 74,
 'á': 75,
 'é': 76,
 'í': 77,
 'ï': 78,
 'ñ': 79,
 'ó': 80,
 'ú': 81,
 'ü': 82}

In [12]:
idx2char 

array(['\n', '\r', ' ', '!', '(', ')', '*', ',', '-', '.', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'Z', '[', ']',
       '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
       'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
       'z', '¡', '«', '»', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'á', 'é', 'í',
       'ï', 'ñ', 'ó', 'ú', 'ü'], dtype='<U1')

##   <span style="color:#4361EE">Transforma el texto en un arreglo de caracteres </span>

In [13]:
text_as_int = np.array([char2idx[c] for c in text])

In [14]:
text_as_int

array([ 1,  0, 23, ...,  0,  1,  0])

In [15]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '(' :   4,
  ')' :   5,
  '*' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


### <span style="color:#4CC9F0">Demostración del uso de los diccionarios</span>

La función `repr` convierte el argumento en una expresión imprimible (cuando es posible).

In [16]:
# Muestre como los primeros 13 caracteres del texto se asignan a números enteros
print ('{} ---- caracteres mapeados a int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

['\r', '\n', 'L', 'O', 'S', ' ', 'C', 'O', 'N', 'S', 'E', 'J', 'O'] ---- caracteres mapeados a int ---- > [ 1  0 23 26 30  2 15 26 25 30 17 22 26]


### <span style="color:#4CC9F0">La tarea de predicción</span>


Dado un caracter, o una secuencia de caracteres, ¿cuál es el próximo caracter más probable? Esta es la tarea en la que vamos a entrenar al modelo. 

La entrada al modelo será una secuencia de caracteres, y entrenamos al modelo para predecir la salida, el siguiente caracter en cada paso de tiempo.

Dado que las RNR mantienen un estado interno que depende de los elementos vistos anteriormente, dados todos los caracteres calculados hasta este momento, ¿cuál es el siguiente caracter?


##   <span style="color:#4361EE">Creación de datos de entrenamiento y etiquetas  </span>

El texto se divide  en secuencias de entrenamiento. Cada secuencia de entrada contendrá  una longitud `seq_length` de caracteres del texto.

Para cada secuencia de entrada, las etiquetas correspondientes contienen la misma longitud del texto, excepto que se desplaza un caracter a la derecha.

Así que primero se divide el texto en trozos de `seq_length + 1`. Por ejemplo, digamos que `seq_length` es `3` y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia de destino  "ola".


In [17]:
# La oración de longitud máxima que queremos para una sola entrada en caracteres
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

Luego se usa la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de índices de caracteres. Un tensor de enteros.

In [18]:
# Crear los ejemplos de entrenamiento / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])




L
O
S
 
C
O
N
S


2022-08-23 22:29:25.156492: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-23 22:29:25.179711: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
char_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

El método por lotes nos permite convertir fácilmente estos caracteres individuales en secuencias del tamaño deseado.


In [20]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'\r\nLOS CONSEJOS DE UN PADRE\r\n\r\n\r\nEl León, el rey de las selvas, agonizaba en el hueco de su caverna...'
'.\r\n\r\nÁ su lado estaba su hijo, el _nuevo león_, el rey futuro de todos los\r\nanimales.\r\n\r\nEl monarca m'
'oribundo le daba penosamente el último consejo, el más\r\nimportante.\r\n\r\n Huye del hombre le decía: huy'
'e siempre; no pretendas luchar con él.\r\n\r\nEres señor absoluto de los demás animales, no los temas; do'
'mínalos,\r\ncastígalos, devóralos si tienes hambre.\r\n\r\nCon todos puedes luchar, á todos puedes vencer; '


In [21]:
sequences

<BatchDataset element_spec=TensorSpec(shape=(101,), dtype=tf.int64, name=None)>

In [22]:
for item in sequences.take(5):
  print(repr(idx2char[item.numpy()]))

array(['\r', '\n', 'L', 'O', 'S', ' ', 'C', 'O', 'N', 'S', 'E', 'J', 'O',
       'S', ' ', 'D', 'E', ' ', 'U', 'N', ' ', 'P', 'A', 'D', 'R', 'E',
       '\r', '\n', '\r', '\n', '\r', '\n', 'E', 'l', ' ', 'L', 'e', 'ó',
       'n', ',', ' ', 'e', 'l', ' ', 'r', 'e', 'y', ' ', 'd', 'e', ' ',
       'l', 'a', 's', ' ', 's', 'e', 'l', 'v', 'a', 's', ',', ' ', 'a',
       'g', 'o', 'n', 'i', 'z', 'a', 'b', 'a', ' ', 'e', 'n', ' ', 'e',
       'l', ' ', 'h', 'u', 'e', 'c', 'o', ' ', 'd', 'e', ' ', 's', 'u',
       ' ', 'c', 'a', 'v', 'e', 'r', 'n', 'a', '.', '.', '.'], dtype='<U1')
array(['.', '\r', '\n', '\r', '\n', 'Á', ' ', 's', 'u', ' ', 'l', 'a',
       'd', 'o', ' ', 'e', 's', 't', 'a', 'b', 'a', ' ', 's', 'u', ' ',
       'h', 'i', 'j', 'o', ',', ' ', 'e', 'l', ' ', '_', 'n', 'u', 'e',
       'v', 'o', ' ', 'l', 'e', 'ó', 'n', '_', ',', ' ', 'e', 'l', ' ',
       'r', 'e', 'y', ' ', 'f', 'u', 't', 'u', 'r', 'o', ' ', 'd', 'e',
       ' ', 't', 'o', 'd', 'o', 's', ' ', 'l', 'o', 's', '


Para cada secuencia, duplíquela y cámbiela para formar el texto de entrada y de destino utilizando el método `map` para aplicar una función simple a cada lote. Es  similar a la función apply de R.

In [23]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [24]:
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [25]:
dataset.take(2)

<TakeDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [26]:
for item in dataset.take(2):
  print(item)

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 1,  0, 23, 26, 30,  2, 15, 26, 25, 30, 17, 22, 26, 30,  2, 16, 17,
        2, 32, 25,  2, 27, 13, 16, 29, 17,  1,  0,  1,  0,  1,  0, 17, 51,
        2, 23, 44, 80, 53,  7,  2, 44, 51,  2, 57, 44, 64,  2, 43, 44,  2,
       51, 40, 58,  2, 58, 44, 51, 61, 40, 58,  7,  2, 40, 46, 54, 53, 48,
       65, 40, 41, 40,  2, 44, 53,  2, 44, 51,  2, 47, 60, 44, 42, 54,  2,
       43, 44,  2, 58, 60,  2, 42, 40, 61, 44, 57, 53, 40,  9,  9])>, <tf.Tensor: shape=(100,), dtype=int64, numpy=
array([ 0, 23, 26, 30,  2, 15, 26, 25, 30, 17, 22, 26, 30,  2, 16, 17,  2,
       32, 25,  2, 27, 13, 16, 29, 17,  1,  0,  1,  0,  1,  0, 17, 51,  2,
       23, 44, 80, 53,  7,  2, 44, 51,  2, 57, 44, 64,  2, 43, 44,  2, 51,
       40, 58,  2, 58, 44, 51, 61, 40, 58,  7,  2, 40, 46, 54, 53, 48, 65,
       40, 41, 40,  2, 44, 53,  2, 44, 51,  2, 47, 60, 44, 42, 54,  2, 43,
       44,  2, 58, 60,  2, 42, 40, 61, 44, 57, 53, 40,  9,  9,  9])>)
(<tf.Tensor: shap

In [27]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\r\nLOS CONSEJOS DE UN PADRE\r\n\r\n\r\nEl León, el rey de las selvas, agonizaba en el hueco de su caverna..'
Target data: '\nLOS CONSEJOS DE UN PADRE\r\n\r\n\r\nEl León, el rey de las selvas, agonizaba en el hueco de su caverna...'


In [28]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 1 ('\r')
  expected output: 0 ('\n')
Step    1
  input: 0 ('\n')
  expected output: 23 ('L')
Step    2
  input: 23 ('L')
  expected output: 26 ('O')
Step    3
  input: 26 ('O')
  expected output: 30 ('S')
Step    4
  input: 30 ('S')
  expected output: 2 (' ')


### <span style="color:#4CC9F0">Creación de lotes de entrenamiento</span>

Usamos `tf.data` para dividir el texto en secuencias manejables. Pero antes de introducir estos datos en el modelo, necesitamos mezclar los datos y empaquetarlos en lotes.

In [29]:
# tamaño del lote
BATCH_SIZE = 64

# Tamaño de búfer para mezclar el conjunto de datos
# (la TF data está diseñado para trabajar con secuencias posiblemente infinitas,
# para que no intente mezclar toda la secuencia en la memoria. En cambio,
# mantiene un búfer en el que mezcla elementos).

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

##   <span style="color:#4361EE">Construcción del Modelo</span>


Use `tf.keras.Sequential` para definir el modelo. Para este sencillo ejemplo, se utilizan tres capas para definir nuestro modelo:

In [30]:
# Tamaño del vocabulario en caracteres
vocab_size = len(vocab)

# Dimensión del embedding
embedding_dim = 256

# Número de unidades de RNR
rnn_units = 1024

In [31]:
vocab_size

83

In [32]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [33]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)


<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Tratamiento_de_Lenguaje_Natural/Imagenes/text_generation_training.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Modelo de generación de texto</p>
</figcaption>
</figure>

Fuente: [TensorFlow](https://www.tensorflow.org/text/tutorials/text_generation#build_the_model)

##   <span style="color:#4361EE">Prueba del modelo</span>


Ahora ejecute el modelo para ver que se comporta como se esperaba.

Primero verifique la forma de la salida:

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 83) # (batch_size, sequence_length, vocab_size)


En el ejemplo anterior, la longitud de secuencia de la entrada es **100** pero el modelo puede ejecutarse en entradas de cualquier longitud.

El dataset de entrenamiento que tiene lotes de tamaño 64 * 100, los cuales tiene orden aleatorio. El primer lote actual y se pasa por el modelo.

El modelo tiene de momento los pesos iniciales. El lote de datos pasa por el modelo y devuelve un tensor de tamaño (64, 100, 83).

Como el vocabulario tiene 83 caracteres y cada secuencia tiene tamaño 100, el modelo predice el siguiente caracter para cada caracter en la entrada.

La predicción funciona así:

Por cada caracter de entrada, se predice el siguiente caracter así: el modelo asigna un valor numérico a cada elemento en el vocabulario. Si se eligiera el máximo el caracter seleccionado sería simplemente el caracter con el valor más grande.

Ya por fuera del modelo.

El siguiente es el vector de predicciones del siguiente caracter para todos los caracteres de la primera secuencia en el primer bloque.

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           21248     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 83)            85075     
                                                                 
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


Para obtener predicciones reales del modelo, necesitamos tomar muestras de la distribución de salida, para obtener índices de caracteres reales. Esta distribución está definida por los [logits](https://es.wikipedia.org/wiki/Logit) sobre el vocabulario de los caracteres.

```{admonition} Nota
Es mejor muestrear esta distribución, que tomar el `argmax` para evitar que el modelo quede atascado en un bucle. 
```

Probemos para el primer ejemplo en el lote:

In [36]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 83), dtype=float32, numpy=
array([[ 7.7511012e-03,  2.1465146e-03, -8.4174350e-03, ...,
        -2.0824226e-03,  3.4331684e-03, -5.3737313e-07],
       [ 2.2362726e-02,  8.3696060e-03, -9.4878953e-03, ...,
        -2.8603021e-03,  1.5719300e-03, -6.9782292e-03],
       [ 8.5425414e-03,  2.9341828e-03, -6.2398179e-03, ...,
         8.0953473e-03,  9.1030542e-04, -2.2933993e-02],
       ...,
       [ 5.7159332e-03, -1.3102829e-03, -1.2524583e-02, ...,
         3.2841796e-03, -6.1105359e-03,  1.9473399e-03],
       [ 9.2278738e-03, -5.5279848e-03,  8.8485423e-05, ...,
        -1.1623040e-02, -8.1506539e-03,  2.4504704e-02],
       [-1.2800302e-03, -1.3847392e-02,  1.2020447e-03, ...,
        -1.0831656e-03, -1.2853913e-02,  1.6620940e-02]], dtype=float32)>

Estos valores son logits para obtener probabilidades. Entonces se normaliza a una distribución categórica, es decir se usa softmax. Con esto, se genera una muestra de la variable categórica para seleccionar el caracter.

El siguiente es el vector de predicciones del siguiente caracter para todos los caracteres de  la primera secuencia en el primer bloque.

Vamos a predecir los caracteres como indicamos.

In [38]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices

<tf.Tensor: shape=(100, 1), dtype=int64, numpy=
array([[52],
       [64],
       [41],
       [17],
       [11],
       [70],
       [66],
       [16],
       [42],
       [23],
       [20],
       [38],
       [33],
       [16],
       [ 8],
       [29],
       [57],
       [21],
       [24],
       [35],
       [56],
       [13],
       [66],
       [59],
       [17],
       [79],
       [72],
       [28],
       [68],
       [51],
       [23],
       [12],
       [17],
       [52],
       [33],
       [66],
       [36],
       [ 0],
       [43],
       [20],
       [ 9],
       [50],
       [25],
       [45],
       [15],
       [30],
       [68],
       [30],
       [52],
       [10],
       [25],
       [25],
       [36],
       [47],
       [12],
       [53],
       [21],
       [62],
       [24],
       [21],
       [70],
       [14],
       [60],
       [ 3],
       [82],
       [ 1],
       [77],
       [56],
       [ 1],
       [69],
       [48],
       [54],
       [13],
   

In [39]:
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([52, 64, 41, 17, 11, 70, 66, 16, 42, 23, 20, 38, 33, 16,  8, 29, 57,
       21, 24, 35, 56, 13, 66, 59, 17, 79, 72, 28, 68, 51, 23, 12, 17, 52,
       33, 66, 36,  0, 43, 20,  9, 50, 25, 45, 15, 30, 68, 30, 52, 10, 25,
       25, 36, 47, 12, 53, 21, 62, 24, 21, 70, 14, 60,  3, 82,  1, 77, 56,
        1, 69, 48, 54, 13, 73, 72, 21, 74, 77, 18, 62, 15, 57, 47, 51, 48,
       58, 82,  3, 44, 43, 57, 69,  6,  4, 34, 64, 52, 49, 64, 41])

Esto nos da en cada paso de tiempo, una predicción del siguiente índice de caracteres:

In [40]:
sampled_indices

array([52, 64, 41, 17, 11, 70, 66, 16, 42, 23, 20, 38, 33, 16,  8, 29, 57,
       21, 24, 35, 56, 13, 66, 59, 17, 79, 72, 28, 68, 51, 23, 12, 17, 52,
       33, 66, 36,  0, 43, 20,  9, 50, 25, 45, 15, 30, 68, 30, 52, 10, 25,
       25, 36, 47, 12, 53, 21, 62, 24, 21, 70, 14, 60,  3, 82,  1, 77, 56,
        1, 69, 48, 54, 13, 73, 72, 21, 74, 77, 18, 62, 15, 57, 47, 51, 48,
       58, 82,  3, 44, 43, 57, 69,  6,  4, 34, 64, 52, 49, 64, 41])

Esta salida significa que el caracter predicho como el siguiente para el primer caracter es el caracter 21 y así sucesivamente.

Los textos lucen así actualmente.

Decodifíquelos para ver el texto predicho por este modelo no entrenado:

In [41]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'del brazo.\r\nPero se había reunido demasiada gente á su alrededor, y la autoridad\r\ntemió que esto fue'

Next Char Predictions: 
 'mybE;Á¡DcLH]VD-RrIMYqA¡tEñÍQ»lL?EmV¡Z\ndH.kNfCS»Sm:NNZh?nIwMIÁBu!ü\ríq\r¿ioAÑÍIÓíFwCrhlisü!edr¿*(Xymjyb'


##   <span style="color:#4361EE">Entrenamiento </span>

En este punto, el problema puede tratarse como un problema de clasificación estándar. Dado el estado RNR anterior, y la entrada en este paso de tiempo, predice la clase del siguiente caracter.

Se define una función de pérdida adecuada para este caso. Esta función recibe los targets (*labels*) y las predicciones. La función calcula la distribución a partir de las predicciones y genera para cada caracter un valor predicho. Esto es lo que se necesita para calcular la entropía cruzada en este caso (promedio). Este es el valor retornado. En el ejemplo, como entra una secuencia y se predicen 100 caracteres hay 100 distribuciones para el siguiente caracter. Uno por cada caracter en la entrada. En total hay un lote de 64 secuencias. Luego la función de pérdida se basa en el promedio de  64∗100 = 6400  entropías cruzadas dispersas.

### <span style="color:#4CC9F0">Adjuntar un optimizador y una función de pérdida</span>

La función de pérdida estándar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque se aplica en la última dimensión de las predicciones.

Debido a que nuestro modelo devuelve logits, debemos establecer el indicador `from_logits`.

In [42]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 83)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4193115



Configure el procedimiento de entrenamiento utilizando el método `tf.keras.Model.compile`. Usaremos `tf.keras.optimizers.Adam` con argumentos predeterminados y la función de pérdida.

In [43]:
model.compile(optimizer='adam', loss=loss)

### <span style="color:#4CC9F0">Configuración de checkpoints</span>

Utilice un `tf.keras.callbacks.ModelCheckpoint` para asegurarse de que los puntos de control se guarden durante el entrenamiento. Se crea un directorio en el cual se guardará los *checkpoints* con el estado del modelo. Los pesos son guardados allí para uso posterior.

In [44]:
# Directorio en donde se guardarán los checkpoints
checkpoint_dir = './training_checkpoints'
# Nombre de los archivos de los  checkpoint 
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### <span style="color:#4CC9F0">Ejecuta el entrenamiento</span>

In [45]:
EPOCHS=10

In [46]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
206/206 [==============================] - 915s 4s/step - loss: 2.5266
Epoch 2/10
206/206 [==============================] - 779s 4s/step - loss: 1.8629
Epoch 3/10
206/206 [==============================] - 775s 4s/step - loss: 1.5435
Epoch 4/10
206/206 [==============================] - 774s 4s/step - loss: 1.2481
Epoch 5/10
206/206 [==============================] - 774s 4s/step - loss: 0.9411
Epoch 6/10
206/206 [==============================] - 772s 4s/step - loss: 0.6339
Epoch 7/10
206/206 [==============================] - 773s 4s/step - loss: 0.4297
Epoch 8/10
206/206 [==============================] - 776s 4s/step - loss: 0.3425
Epoch 9/10
206/206 [==============================] - 769s 4s/step - loss: 0.3080
Epoch 10/10
206/206 [==============================] - 767s 4s/step - loss: 0.2895


En `history` queda la información de `loss`, y `acurracy` para revisión.

##   <span style="color:#4361EE">Generación de texto </span>

### <span style="color:#4CC9F0">Restaura el último checkpoint</span>

En esta sección vamos a tomar un preentrenamiento. 

Usaremos los pesos almacenados, pero ahora vamos a cambiar el modelo para recibir lotes de tamaño 1. Es decir vamos a recibir una secuencia para hacer la predicción a partir de esa secuencia.


Debido a la forma en que se pasa el estado RNR de un paso a otro, el modelo solo acepta un tamaño de lote fijo una vez construido.

Para ejecutar el modelo con un tamaño de lote diferente, necesitamos reconstruir el modelo y restaurar los pesos desde el punto de control.

In [47]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            21248     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 83)             85075     
                                                                 
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


### <span style="color:#4CC9F0">El bucle de predicción</span>

El siguiente bloque de código genera el texto:

* Comienza eligiendo una cadena de inicio, inicializando el estado de la  RNR y configurando el número de caracteres a generar.

* Obtenga la distribución de predicción del siguiente caracter utilizando la cadena de inicio y el estado de la  RNR.

* Luego, use una distribución categórica para calcular el índice del caracter predicho. Use este caracter predicho como nuestra próxima entrada al modelo.

* El estado de la RNR devuelto por el modelo se retroalimenta al modelo para que ahora tenga más contexto, en lugar de una sola palabra. Después de predecir la siguiente palabra, los estados RNR modificados se retroalimentan nuevamente en el modelo, que es como aprende a medida que obtiene más contexto de las palabras predichas previamente.

La siguiente es la línea que define la capa GRU arriba en la definición del modelo.

In [ ]:
#tf.keras.layers.GRU(rnn_units,  return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')

El parámetro `stateful` determina si los valores de estado recurrente deben mantenerse `True` o no, cuando se pasa al siguiente caracter. En este caso, al mantenerse esos valores, se mantiene la memoria de la secuencia inicial y de los caracteres que van siendo generados. 

Por eso, solamente se pasa en el primer paso la secuencia de entrada completa. Desde el segundo paso solamente se pasa el nuevo caracter (que es el predicho). Con esta predicción y la memoria del resto de la secuencia anterior mantenida en estado recurrente se hace la siguiente predicción y así sucesivamente tomado la última secuencia pasada por la capa.

Para pasar la última secuencia tratada en la capa GRU, se usa el parámetro `return_sequences`. Por defecto es `False`, es decir se omite en la salida la última secuencia pasada por la capa GRU. Aquí hemos colocado `return_sequences=True`, para pasar únicamente el último carater predicho.



Al observar el texto generado, verá que el modelo sabe cuándo colocar mayúsculas, hacer párrafos e imita un vocabulario de escritura similar a Shakespeare. Con el pequeño número de épocas de entrenamiento, aún no ha aprendido a formar oraciones coherentes.

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Libro-Fundamentos/main/Tratamiento_de_Lenguaje_Natural/Imagenes/text_generation_sampling.png" width="500" height="500" align="center"/>
</center>
<figcaption>
<p style="text-align:center">Muestreo  de palabras</p>
</figcaption>
</figure>

Fuente: [TensorFlow](https://www.tensorflow.org/text/tutorials/text_generation#generate_text)

La siguiente función implementa el ciclo de predicción:

In [49]:
def generate_text(model, start_string):
    # Paso de evaluación (generación de texto usando el modelo aprendido)

    # Número de caracteres a generar
    num_generate = 1000

    # Convertir nuestra cadena de inicio en números (vectorizar)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Cadena vacía para almacenar nuestros resultados
    text_generated = []

    # Las bajas temperaturas dan como resultado un texto más predecible.
    # Las temperaturas más altas dan como resultado un texto más sorprendente.
    # Experimente para encontrar la mejor configuración.
    temperature = 1.0

    # Aquí batch size == 1
    model.reset_states() # borra las unidades recurrentes
    for i in range(num_generate):
        predictions = model(input_eval)
        # eliminar la dimensión del lote
        predictions = tf.squeeze(predictions, 0)

        # usar una distribución categórica para predecir la palabra devuelta por el modelo
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pasamos la palabra predicha como siguiente entrada al modelo
        # junto con el estado oculto anterior
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

### <span style="color:#4CC9F0">Notas</span>

1. `reset_states` borra solo los estados recurrentes de la red. Vale la pena mencionar que dependiendo de si la opción `stateful = True` se configuró en la red, el comportamiento de esta función podría ser diferente. Si no está configurado, todos los estados se restablecen automáticamente después de cada cálculo por lotes en su red (por ejemplo, después de llamar a `fit`, `predict` y `evaluate` también). Si no es así, debe llamar a `reset_states` cada vez que desee que las llamadas del modelo consecutivas sean independientes.
2. `tf.expand_dims` devuelve un tensor con una dimensión adicional insertada en el eje del índice.

In [50]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: CASAntr
    Un confesonario fino.

Dedrándose en partecilla tropezón, se queda otra, llamada María, que es muy amiga del Boti, si el señor
Frutos. ¡La
noche, que se parece que taña
    Una lanchita con repararnos!_

An la talde su dese arrastrándose de aquel sitio, pero en dando de Fuencar, única propiedad que no
tenía hipotecada. Allí trances altas tornstantes y bienhechores compañeros del oro, que pe
acometidad de la niña, y una lluvia y después le cuento en una
palamiento.

 No cabellos casi el blanco ni el fondo de us de un murmitino:

 Aquéllas que brillaba entre los l Pues, asegurando que el
que con ahoras, y
lo fintorres lágrimas calicantes una ballena cuando ardon


Un hombre de Casilidad, llamado Consejo reflexionaba como un reyobutaban, pagaráje, y como impuso en tono imperativo:

 ¡Dos billetes de primera para París...!

MY oyermás que no han dado las
tres, aunque ya faltaron todos á una y otra oreja de la noche.... Si tu misma edad, seis años.... La


Lo más fácil que puedes hacer para mejorar los resultados es entrenarlo por más tiempo (prueba `EPOCHS = 30`).

También puede experimentar con una cadena de inicio diferente, o intentar agregar otra capa RNR para mejorar la precisión del modelo, o ajustar el parámetro de temperatura para generar predicciones más o menos aleatorias.

##  <span style="color:#4361EE">Entrenamiento personalizado </span>



El procedimiento de entrenamiento anterior es simple, pero no le da mucho control.

Entonces, ahora que ha visto cómo ejecutar el modelo manualmente, descomprimimos el ciclo de entrenamiento e implementémoslo nosotros mismos. Esto proporciona un punto de partida, si por ejemplo, se implementa *aprendizaje curricular* para ayudar a estabilizar la salida de bucle abierto del modelo.

Usaremos `tf.GradientTape` para rastrear los gradientes. Puede obtener más información sobre este enfoque leyendo en [diferenciación automática](https://www.tensorflow.org/guide/basics#automatic_differentiation).

El procedimiento funciona de la siguiente manera:

* Primero, inicialice el estado RNR. Hacemos esto llamando al método `tf.keras.Model.reset_states`.

* Luego, repita el conjunto de datos (lote por lote) y calcule las *predicciones* asociadas con cada una.

* Abra un `tf.GradientTape`, y calcule las predicciones y pérdidas en ese contexto.

* Calcular los gradientes de la pérdida con respecto a las variables del modelo utilizando el método `tf.GradientTape.grads`.

* Finalmente, dé un paso hacia abajo utilizando el método `tf.train.Optimizer.apply_gradients` del optimizador.

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [52]:
optimizer = tf.keras.optimizers.Adam()

In [53]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [54]:
# Paso de entrenamiento
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # inicializando el estado oculto al comienzo de cada época
    # inicialmente oculto es Ninguno
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

    if batch_n % 100 == 0:
        template = 'Epoch {} Batch {} Loss {}'
        print(template.format(epoch+1, batch_n, loss))
 
    # guardar (punto de control) el modelo cada 5 épocas
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Loss 2.0439
Time taken for 1 epoch 813.6043629646301 sec

Epoch 2 Loss 1.6835
Time taken for 1 epoch 814.6451671123505 sec

Epoch 3 Loss 1.4132
Time taken for 1 epoch 838.6421868801117 sec

Epoch 4 Loss 1.1056
Time taken for 1 epoch 824.6420001983643 sec

Epoch 5 Loss 0.8184
Time taken for 1 epoch 749.619856595993 sec

Epoch 6 Loss 0.5499
Time taken for 1 epoch 751.6419911384583 sec

Epoch 7 Loss 0.3963
Time taken for 1 epoch 749.1919178962708 sec

Epoch 8 Loss 0.3423
Time taken for 1 epoch 750.258998632431 sec

Epoch 9 Loss 0.3192
Time taken for 1 epoch 747.5328431129456 sec

Epoch 10 Loss 0.2823
Time taken for 1 epoch 830.8155710697174 sec



##   <span style="color:#4361EE">Referencias</span>

1. [Tensorflow, Text generation with a RNN](https://www.tensorflow.org/tutorials/text/text_generation)
1. [Ralf C. Staudemeyer and Eric Rothstein Morris,Understanding LSTM a tutorial into Long Short-Term Memory Recurrent Neural Networks*, arxiv, September 2019](https://arxiv.org/pdf/1909.09586.pdf)
1. [Karpathy, The Unreasonable Effectiveness of Recurrent Neural Networks](  http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
1. [Proyecto Gutemberg](https://www.gutenberg.org/)